In [1]:
import pandas as pd
import numpy as np
import scipy
import statsmodels.stats.multitest as smm
from statsmodels.sandbox.stats.multicomp import multipletests 

import warnings
warnings.filterwarnings('ignore')

In [2]:
def print_to_file(value, name):
    print(name, value)
    with open(name, "w") as file:
        file.write(str(value))

# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

## В этом задании вы:

- вспомните, что такое t-критерий Стьюдента и для чего он применяется
- сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
- почувствуете разницу в результатах применения различных методов поправки на множественную проверку

## Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

In [3]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')
df.head(5)

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

## Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$F_c(C,T) =
\left\{\begin{matrix}\frac{T}{C}, T>C
\\ \frac{-C}{T}, T<C
\end{matrix}\right.$$

где $C, T$ — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

## Инструкции к решению задачи

Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.
### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

- для групп __normal (control)__ и __early neoplasia (treatment)__
- для групп __early neoplasia (control)__ и __cancer (treatment)__

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [4]:
df_normal = df[df.Diagnosis == 'normal']
df_early = df[df.Diagnosis == 'early neoplasia']
df_cancer = df[df.Diagnosis == 'cancer']

In [5]:
def compare_df_ttest_columns(df_control, df_treatment, cols):
    p_vals = [scipy.stats.ttest_ind(df_control[col], df_treatment[col], equal_var = False).pvalue for col in cols]
    return np.array(p_vals)

In [6]:
cols = df_normal.columns[2:].to_numpy()
p_vals_normal_early = compare_df_ttest_columns(df_normal, df_early, cols)
p_vals_early_cancer = compare_df_ttest_columns(df_early, df_cancer, cols)

In [7]:
t = 0.05
diffs_normal_early = sum(np.array(p_vals_normal_early) < t)
diffs_early_cancer = sum(np.array(p_vals_early_cancer) < t)
print_to_file(diffs_normal_early,'1-1-normal_early.txt')
print_to_file(diffs_early_cancer,'1-2-early_cancer.txt')

1-1-normal_early.txt 1575
1-2-early_cancer.txt 3490


Это достаточно огромное количество признаков. 

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль __multitest__ из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.

__Обратите внимание, что__

- применять поправку на множественную проверку нужно __ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия__.
- при использовании поправки на уровне значимости 0.025 __меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия__ (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)! 

In [8]:
def get_fold_change(C,T):
    if T>C:
        return T/C
    else:
        return -C/T
    
def get_fold_changes(df_control, df_treatment, cols, p_vals, method, alpha):
    reject = multipletests(p_vals, alpha = alpha, method = method) [0]
    means_control = df_control.loc[:,cols[reject]].mean()
    means_treatment = df_treatment.loc[:,cols[reject]].mean()
    df_means = pd.DataFrame([means_control, means_treatment])
    fold_changes = df_means.apply(lambda cols: get_fold_change(cols[0], cols[1]))
    return fold_changes

def get_practic_fold_change_count(df_control, df_treatment, cols, 
                                  p_vals, method, alpha, threshold):
    fold_changes = get_fold_changes(df_control, df_treatment, cols, p_vals, method, alpha)
    return np.sum(np.abs(fold_changes)>threshold)

In [9]:
alpha = 0.05 / 2
practic_fc_normal_early = get_practic_fold_change_count(df_normal, df_early, cols, 
                                                        p_vals_normal_early, 'holm', alpha, 1.5)
practic_fc_early_cancer = get_practic_fold_change_count(df_early, df_cancer, cols, 
                                                        p_vals_early_cancer, 'holm', alpha, 1.5)

print_to_file(practic_fc_normal_early,'2-1-normal_early-holm.txt')
print_to_file(practic_fc_early_cancer,'2-2-early_cancer-holm.txt')

2-1-normal_early-holm.txt 2
2-2-early_cancer-holm.txt 77


### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

In [10]:
alpha = 0.05 / 2
practic_fc_normal_early = get_practic_fold_change_count(df_normal, df_early, cols, 
                                                        p_vals_normal_early, 'fdr_bh', alpha, 1.5)
practic_fc_early_cancer = get_practic_fold_change_count(df_early, df_cancer, cols, 
                                                        p_vals_early_cancer, 'fdr_bh', alpha, 1.5)

print_to_file(practic_fc_normal_early,'3-1-normal_early-fdr_bh.txt')
print_to_file(practic_fc_early_cancer,'3-2-early_cancer-fdr_bh.txt')

3-1-normal_early-fdr_bh.txt 4
3-2-early_cancer-fdr_bh.txt 524


## Выводы
Мы сравнивали контрольную и тестовую группы. С помощью t-критерия сравнили средние признаков из этих двух выборок - получили p-value признаков. 
Так как выборки две, мы использовали поправки Холма и Бенджамини-Хохберга. Они дали нам дали статистически значимые отличия, на которых мы применили fold_change для определения практической значимости. Таким образом мы отобрали только те признаки, которые имеют статистически и практически значимые отличия.
Метод Холма дал меньшее количество значимо отличающихся признаков, чем метод Бенджамини-Хохберга. Метод Холма избегает всех ошибок первого рода, но, увы, не смотрит за ошибками второго рода. Метод Бенджамини-Хохберга находит баланс между ошибками первого и второго родов.